In [1]:
import torch
import torch.onnx
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.1 MB/s eta 0:00:00


In [2]:
# Создание модели LSTM
model = torch.nn.LSTM(10, 20, 2)
dummy_input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = model(dummy_input, (h0, c0))
print(output)

Результат запуска модели LSTM в PyTorch: tensor([[[ 0.1258, -0.3895, -0.3246, -0.0965, -0.0983, -0.3848, -0.0813,
           0.1429,  0.2157,  0.4254,  0.4932, -0.2771,  0.0766,  0.4091,
          -0.1410, -0.1197, -0.0789,  0.3550, -0.3892, -0.3821],
         [ 0.0742, -0.1467, -0.2451,  0.1934, -0.3341, -0.0287,  0.2274,
          -0.4068, -0.0855,  0.3704,  0.2240,  0.3621, -0.1172,  0.0362,
           0.3067,  0.2389,  0.3799,  0.2139,  0.2701,  0.2606],
         [-0.1030,  0.0844,  0.0249, -0.1294, -0.1087,  0.1967, -0.2009,
           0.1484,  0.1211,  0.2803, -0.2229, -0.0185, -0.1268, -0.1463,
          -0.1710,  0.0240, -0.0095, -0.0333, -0.0355, -0.2336]],

        [[-0.0591, -0.2219, -0.1045, -0.1148, -0.1265, -0.1479,  0.0686,
           0.0678,  0.1019,  0.1614,  0.2672, -0.1546,  0.0757,  0.1740,
          -0.0614, -0.0183, -0.0208,  0.2660, -0.0467, -0.1584],
         [-0.0286, -0.1778, -0.0420,  0.0642, -0.1556, -0.1333,  0.1062,
          -0.2172, -0.0766,  0.1221,  0.

In [3]:
# Конвертация в ONNX
torch.onnx.export(model, dummy_input, "lstm.onnx")

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4661: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [4]:
# Загрузка и запуск модели в ONNX
import onnx
import onnxruntime
onnx_model = onnx.load("lstm.onnx")
ort_session = onnxruntime.InferenceSession("lstm.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)

Результат запуска модели LSTM в ONNX: [array([[[-5.46830259e-02, -4.52855863e-02,  3.48897688e-02,
          2.27431264e-02, -4.06667553e-02, -6.82506487e-02,
          1.00012191e-01,  1.01888855e-03, -1.30332308e-02,
         -8.07583034e-02, -8.28616917e-02, -1.39922686e-02,
         -1.05513902e-02,  1.11040268e-02, -3.57686430e-02,
          5.28107509e-02,  4.82518375e-02,  8.63242075e-02,
          4.10928726e-02,  6.59039766e-02],
        [-5.97724579e-02, -2.69594844e-02,  8.33477452e-02,
         -2.00508721e-02, -4.85023409e-02, -4.65546772e-02,
          8.58875737e-02,  2.01797690e-02,  8.64521228e-03,
         -6.84860945e-02, -6.31917566e-02,  2.63656955e-03,
          2.19512060e-02, -3.27748396e-02, -4.75836694e-02,
          3.58438268e-02,  7.75338337e-02,  7.45876282e-02,
          6.35558814e-02,  7.43720531e-02],
        [-5.33675440e-02, -1.97252966e-02,  5.72487153e-02,
          2.80200373e-02, -4.59049977e-02, -6.27448261e-02,
          8.58034343e-02,  1.1338

In [5]:
# Обратная конвертация
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("lstm.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 917.6 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

NotImplementedError: Converter is not implemented (OperationDescription(domain='', operation_type='LSTM', version=14))